## Data Import

In [8]:
library(dplyr)
library(arules)

bookings <- read.csv("output.csv")
head(bookings)

ERROR: Error in library(arules): there is no package called ‘arules’


In [ ]:
# Discretize lead_time and adr to (low, medium, high)

bookings <- discretizeDF(bookings, methods = list(
  lead_time = list(method = "frequency", breaks = 3, 
    labels = c("Low", "Medium", "High")),
  adr = list(method = "frequency", breaks = 3, 
    labels = c("Low", "Medium", "High")),
  stays_in_weekend_nights = list(method = "frequency", breaks = 3, 
    labels = c("Low", "Medium", "High")),
  stays_in_week_nights = list(method = "frequency", breaks = 3, 
    labels = c("Low", "Medium", "High")),
  adults = list(method = "frequency",  breaks = 3, 
    labels = c("Low", "Medium", "High"))
  ),
  default = list(method = "none")
  )
head(bookings)


In [ ]:
index <- 1:ncol(bookings)
bookings[ , index] <- lapply(bookings[ , index], as.factor)
str(bookings)

In [ ]:
head(bookings)

## Transactions

In [ ]:
trans <- as(bookings, "transactions")
image(trans)
summary(trans)

str(bookings)
str(trans)

## Apriori

In [ ]:
rules <- apriori(trans,parameter=list(minlen=2, support=.005, confidence=.9), control = list(verbose=F),appearance = list(rhs=c("adr=Low","adr=Medium","adr=High"),default="lhs"))

rules_lift <- sort(rules, by="lift")
rules_pruned <- rules_lift[!is.redundant(rules_lift, measure="lift")]
length(rules_pruned)

In [ ]:
inspect(rules_pruned,by="lift")


## Plots

In [ ]:
library(arulesViz)

In [ ]:
plot(rules_pruned, method="grouped")

In [ ]:
plot(rules_pruned, method="graph")

In [ ]:
plot(rules_pruned, method="grouped matrix")

In [ ]:
plot(rules_pruned, method="matrix3D")

In [ ]:
plot(rules_pruned, method="paracoord")

In [ ]:
plot(rules_pruned, measure=c("support", "lift"), shading = "confidence")

In [ ]:
top10Rules <- head(rules_pruned, n = 10, by = "confidence")
plot(top10Rules, method = "graph",  engine = "htmlwidget")
saveAsGraph(head(rules_pruned, n = 100, by = "lift"), file = "rules.graphml")

## SubRules 

In [ ]:
subrules <- rules_pruned[quality(rules_pruned)$lift > 2.95]

In [ ]:
subrules

In [ ]:
plot(subrules, method="grouped")


In [ ]:
inspect(subrules,by="lift")
